<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Network-in-Network-(NiN)-BLOCK" data-toc-modified-id="Network-in-Network-(NiN)-BLOCK-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Network in Network (NiN) BLOCK</a></span></li></ul></div>

In [1]:
import torch
from torchsummary import summary
import torch.nn.functional as F
from torch import nn

##  Network in Network (NiN) BLOCK
<img src='../images/nin.jpg'/>
The NiN block consists of one convolutional layer followed by two 1 × 1 convolutional layers that
act as per-pixel fully-connected layers with ReLU activations. The convolution width of the first
layer is typically set by the user. The subsequent widths are fixed to 1 × 1

In [2]:
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU())

In [3]:
nin=nn.Sequential(
  nin_block(in_channels=1,out_channels=96,kernel_size=11,strides=4,padding=0),
    nn.MaxPool2d(kernel_size=3,stride=2),
    nin_block(in_channels=96,out_channels=256,kernel_size=5,strides=1,padding=2),
    nn.MaxPool2d(kernel_size=3,stride=2),
    nin_block(in_channels=256,out_channels=384,kernel_size=5,strides=1,padding=1),
    nn.MaxPool2d(kernel_size=3,stride=2),
    nn.Dropout(0.5),
    nin_block(in_channels=384,out_channels=10,kernel_size=3,strides=1,padding=2),
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten()
    
)

In [5]:
X = torch.randn(size=(1, 1, 224, 224))
for layer in nin:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 10, 10])
MaxPool2d output shape:	 torch.Size([1, 384, 4, 4])
Dropout output shape:	 torch.Size([1, 384, 4, 4])
Sequential output shape:	 torch.Size([1, 10, 6, 6])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [6]:
X = torch.randn(size=(1, 1, 224, 224))
summary(nin,X)

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 96, 54, 54]          --
|    └─Conv2d: 2-1                       [-1, 96, 54, 54]          11,712
|    └─ReLU: 2-2                         [-1, 96, 54, 54]          --
|    └─Conv2d: 2-3                       [-1, 96, 54, 54]          9,312
|    └─ReLU: 2-4                         [-1, 96, 54, 54]          --
|    └─Conv2d: 2-5                       [-1, 96, 54, 54]          9,312
|    └─ReLU: 2-6                         [-1, 96, 54, 54]          --
├─MaxPool2d: 1-2                         [-1, 96, 26, 26]          --
├─Sequential: 1-3                        [-1, 256, 26, 26]         --
|    └─Conv2d: 2-7                       [-1, 256, 26, 26]         614,656
|    └─ReLU: 2-8                         [-1, 256, 26, 26]         --
|    └─Conv2d: 2-9                       [-1, 256, 26, 26]         65,792
|    └─ReLU: 2-10                        [-1, 256, 26, 26]        

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 96, 54, 54]          --
|    └─Conv2d: 2-1                       [-1, 96, 54, 54]          11,712
|    └─ReLU: 2-2                         [-1, 96, 54, 54]          --
|    └─Conv2d: 2-3                       [-1, 96, 54, 54]          9,312
|    └─ReLU: 2-4                         [-1, 96, 54, 54]          --
|    └─Conv2d: 2-5                       [-1, 96, 54, 54]          9,312
|    └─ReLU: 2-6                         [-1, 96, 54, 54]          --
├─MaxPool2d: 1-2                         [-1, 96, 26, 26]          --
├─Sequential: 1-3                        [-1, 256, 26, 26]         --
|    └─Conv2d: 2-7                       [-1, 256, 26, 26]         614,656
|    └─ReLU: 2-8                         [-1, 256, 26, 26]         --
|    └─Conv2d: 2-9                       [-1, 256, 26, 26]         65,792
|    └─ReLU: 2-10                        [-1, 256, 26, 26]        